In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
from tensorflow import keras
import tensorflow as tf
import os
from PIL import Image
from io import BytesIO
from skimage import io
from sklearn.utils import class_weight

tf.config.experimental.list_physical_devices('GPU')


In [ ]:
train_path = r"./Data/nov2019_train"
test_path = r"./Data/nov2019_test"
label_train = r"./Data/label_nov2019_train.txt"
BATCH_SIZE = 128
EPOCHS = 100
class_weight = {0: 1.95454992,
                1: 0.67187447}

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten,MaxPooling2D,Dropout#create model
def import_data(path): 
    count = 0
    x = []
    for filename in os.listdir(path):
        img = Image.open(os.path.join(path,filename))
        img=img.convert('L')
        img = np.array(img)        
        x.append(img)
        count +=1
    x = np.array(x)
    x = x.reshape(count,80,80,1)
    x = x.astype(np.float32)
    return x

def get_untrained_model():
    model = Sequential()#add model layers
    model.add(Conv2D(64,7, activation='relu',padding="same", input_shape=[80,80,1]))
    model.add(MaxPooling2D(2))
    model.add(Conv2D(128,3, activation='relu',padding="same"))
    model.add(Conv2D(128,3, activation='relu',padding="same"))
    model.add(MaxPooling2D(2))
    model.add(Conv2D(256, 3, activation='relu',padding="same"))
    model.add(Conv2D(256, 3, activation='relu',padding="same"))
    model.add(MaxPooling2D(2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def model_cross_validation(n_splits, random_state):
    kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=7)
    cvscores = []
    y_train_label = to_categorical(y_train)

    for train, validation in kfold.split(X_train, y_train):
        model = get_untrained_model()
        model.fit(X_train[train], y_train_label[train], epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1,callbacks=callbacks,class_weight=class_weight)
        scores = model.evaluate(X_train[validation], y_train_label[validation], verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        cvscores.append(scores[1] * 100)
    # serialize weights to HDF5
    model.save_weights("model.h5")
    print("Saved model to disk")

    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


In [ ]:
file_label = open(label_train,"r+")
y_train = [item for item in file_label.read().splitlines()]
from tensorflow.keras.utils import to_categorical#one-hot encode target column


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

In [ ]:
X_train = import_data(train_path)


In [ ]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping
#tf.debugging.set_log_device_placement(True)

gpus = tf.config.experimental.list_logical_devices('GPU')
callbacks = [EarlyStopping(monitor='loss', patience=10, verbose=0)]

if gpus:
  # Replicate your computation on multiple GPUs
  for gpu in gpus:
    with tf.device(gpu.name):
        print(gpu.name)
        model_cross_validation(4, 7)
else :
    model_cross_validation(4, 7)

In [ ]:
from tensorflow.keras.models import load_model

X_test = import_data(test_path)
print(X_test.shape)
model = get_untrained_model()
model.load_weights('model.h5')
model.summary()

In [ ]:
X_test = X_test
result = model.predict_classes(X_test,batch_size=BATCH_SIZE, verbose=0)

In [ ]:
with open("result_predict.txt","w") as f:
    for i in range(len(result)):
        f.write("%i\n" % (result[i]))
        
print(result.shape)
print(result)